# 数据进行合并

In [1]:
import time
import pandas as pd
import numpy as np

# 读取数据
part_1 = pd.read_csv('./input/meinian_round1_data_part1_20180408.txt',sep='$')
part_2 = pd.read_csv('./input/meinian_round1_data_part2_20180408.txt',sep='$')
part_1_2 = pd.concat([part_1,part_2])
part_1_2 = pd.DataFrame(part_1_2).sort_values('vid').reset_index(drop=True)
begin_time = time.time()
print('begin')
# 重复数据的拼接操作
def merge_table(df):
    df['field_results'] = df['field_results'].astype(str)
    if df.shape[0] > 1:
        merge_df = " ".join(list(df['field_results']))
    else:
        merge_df = df['field_results'].values[0]
    return merge_df
# 数据简单处理
print('find_is_copy')
print(part_1_2.shape)
is_happen = part_1_2.groupby(['vid','table_id']).size().reset_index()
# 重塑index用来去重
is_happen['new_index'] = is_happen['vid'] + '_' + is_happen['table_id']
is_happen_new = is_happen[is_happen[0]>1]['new_index']

part_1_2['new_index'] = part_1_2['vid'] + '_' + part_1_2['table_id']

unique_part = part_1_2[part_1_2['new_index'].isin(list(is_happen_new))]
unique_part = unique_part.sort_values(['vid','table_id'])
no_unique_part = part_1_2[~part_1_2['new_index'].isin(list(is_happen_new))]
print('begin')
part_1_2_not_unique = unique_part.groupby(['vid','table_id']).apply(merge_table).reset_index()
part_1_2_not_unique.rename(columns={0:'field_results'},inplace=True)
print('xxx')
tmp = pd.concat([part_1_2_not_unique,no_unique_part[['vid','table_id','field_results']]])
# 行列转换
print('finish')
tmp = tmp.pivot(index='vid',values='field_results',columns='table_id')
tmp.to_csv('./tmp.csv')
print(tmp.shape)
print('totle time',time.time() - begin_time)

begin
find_is_copy
(8104368, 3)
begin
xxx
finish
(57298, 2795)
totle time 136.9307188987732


In [2]:
# 清理缺失值大于阈值的函数
def remain_feat(df,thresh=0.9):
    exclude_feats = []
    print('----------移除数据缺失多的字段-----------')
    print('移除之前总的字段数量',len(df.columns))
    num_rows = df.shape[0]
    for c in df.columns:
        num_missing = df[c].isnull().sum()
        if num_missing == 0:
            continue
        missing_percent = num_missing / float(num_rows)
        if missing_percent > thresh:
            exclude_feats.append(c)
    print("移除缺失数据的字段数量: %s" % len(exclude_feats))
    # 保留超过阈值的特征
    feats = []
    for c in df.columns:
        if c not in exclude_feats:
            feats.append(c)
    print('剩余的字段数量',len(feats))

    df = df[feats]
    return df

threshold = 0.95
tmp = remain_feat(tmp, threshold)

----------移除数据缺失多的字段-----------
移除之前总的字段数量 2795
移除缺失数据的字段数量: 2415
剩余的字段数量 380


# 找出数字特征，进一步筛选

In [3]:
# 找出数字特征，30%以上是数字认为是数字
def is_num(x):
    try:
        float(x)
        if(x == None):
            x = 0
        elif (x != x):
            x = 0
        else:
            x = 1
    except:
        x = 0
    return x

def find_nums(df, threshold=0.3):
    num_feature = []
    print('总特征数：', df.shape[1])
    for c in df.columns:
        df_mask = df[c].apply(is_num)
        if(df_mask.sum() / df[c].count() > threshold):
            num_feature.append(c)
    print('数字特征数量：', len(num_feature))
    return num_feature

def nums_df(df, remain_th, threshold=0.3):
    num_feature = find_nums(df, threshold)
    df_nums = remain_feat(df[num_feature], remain_th)
    return df_nums

num_remain_th = 0.8
# 数值型特征缺失值少于大于0.8删除， 30%以上是数字认为是数字
tmp_nums = nums_df(tmp, num_remain_th, 0.3)

总特征数： 380
数字特征数量： 219
----------移除数据缺失多的字段-----------
移除之前总的字段数量 219
移除缺失数据的字段数量: 147
剩余的字段数量 72


# 对数字型特征进行清洗

In [4]:
import re

# 对于有数字和其他值的，把第一个出现的数字切出来
def search_num(x):
    x_str = str(x)
    pattern = re.compile("\d+\.?\d*")
    match = re.search(pattern, x_str)
    if(match != None):
        return match.group()
    else:
        return x

# 对于第0列，过缓小于50，过速度大于110
def apply_0(x):
    x_str = str(x)
    if('过缓' in x_str):
        x = 40
    elif('过速' in x_str):
        x = 110
    return x

# 切片找到数值
tmp_nums = tmp_nums.applymap(search_num)
# 对第0列进行特殊处理：
tmp_nums.iloc[:,0] = tmp_nums.iloc[:,0].apply(apply_0)

##  缺失值和离群点处理

* 1、含表示正常的cell，填充均值
* 2、大于3倍均值的离群点，可能是误输入，设为3倍均值
* 3、对于缺失值置0，用nmf进行填充


In [36]:
# 判断是是否有数字，用于计算有多少个数字
def has_value(x):
    if x != x:
        return 0
    if x == None:
        return 0
    try:
        float(x)
        return 1
    except:
        return 0
    
# 有数字返回数字，用于求含nan的平均值
def re_value(x):
    if x != x:
        return 0
    if x == None:
        return 0
    try:
        x = float(x)
        return x
    except:
        return 0

# 用于计算方差
def p_fang(c_mean):
    def re_fang(x):
        if x != x:
            return 0
        if x == None:
            return 0
        try:
            float(x)
            return (x-c_mean)*(x-c_mean)
        except:
            return 0
    return re_fang

# 计算均值
def cal_mean(df, ci):
    s = df.iloc[:,ci].apply(re_value).sum()
    n = df.iloc[:,ci].apply(has_value).sum()
    return s/n

# 清除离群点
def clean_out_dot(c_mean, n):
    def out_dot_apply(x):
        try:
            x = float(x)
            if(x > n*c_mean):
                x = n*c_mean
            return x
        except:
            return x
    return out_dot_apply

# 正常值赋均值
def normal_apply(normal_list, c_mean):
    def normal_app(x):
        x = str(x)
        for nor in normal_list:
            if(nor in x):
                x = c_mean
                break
        return x
    return normal_app

# 不能处理的点赋值nan
def nan_apply(x):
    try:
        x = float(x)
    except:
        x = np.nan
    return x

##  清除离群点，正常值赋均值

In [37]:
def out_dot(df, normal_list, n=3):
    # 清除离群点：
    print('清除离群点前100007最大值', df['100007'].apply(re_value).max())
    for ci in range(df.shape[1]):
        c_mean = cal_mean(df, ci)
        df.iloc[:, ci] = df.iloc[:, ci].apply(clean_out_dot(c_mean, n))
    print('清除离群点后100007最大值', df['100007'].apply(re_value).max())
    
    # 正常人填充平均值
    for ci in range(df.shape[1]):
        c_mean = cal_mean(df, ci)
        df.iloc[:, ci] = df.iloc[:, ci].apply(normal_apply(normal_list, c_mean))
        # 其他不能处理类型赋空
        df.iloc[:, ci] = df.iloc[:, ci].apply(nan_apply)
        
    return df

normal_list = ['阴性','正常','Normal','未见异常']
# 大于3倍均值，令它为3倍均值
tmp_nums = out_dot(tmp_nums, normal_list, 3) 

清除离群点前100007最大值 51.8
清除离群点后100007最大值 2.01404259554


# 清除缺失特征太多的人

In [41]:
# 读取train和test集
train_df=pd.read_csv('./input/meinian_round1_train_20180408.csv',sep=',',encoding='gbk')
test_df=pd.read_csv('./input/meinian_round1_test_a_20180409.csv',sep=',',encoding='gbk')

### 清洗train集的label

In [43]:
def clean_label(x):
    x=str(x)
    if '+' in x:#16.04++
        i=x.index('+')
        x=x[0:i]
    if '>' in x:#> 11.00
        i=x.index('>')
        x=x[i+1:]
    if len(x.split(sep='.'))>2:#2.2.8
        i=x.rindex('.')
        x=x[0:i]+x[i+1:]
    if '未做' in x or '未查' in x or '弃查' in x or '不查' in x or '弃检' in x:
        x=np.nan
    if str(x).isdigit()==False and len(str(x))>4:
        x=x[0:4]
    return x

def clean_label(x):
    x=str(x)
    if '+' in x:#16.04++
        i=x.index('+')
        x=x[0:i]
    if '>' in x:#> 11.00
        i=x.index('>')
        x=x[i+1:]
    if len(x.split(sep='.'))>2:#2.2.8
        i=x.rindex('.')
        x=x[0:i]+x[i+1:]
    if '未做' in x or '未查' in x or '弃查' in x or '不查' in x or '弃检' in x:
        x=np.nan
    if str(x).isdigit()==False and len(str(x))>4:
        x=x[0:4]
    return x

def data_clean(df):
    label_list = ['收缩压','舒张压','血清甘油三酯','血清高密度脂蛋白','血清低密度脂蛋白']
    for c in label_list:
        df[c]=df[c].apply(clean_label)
        df[c]=df[c].astype('float64')
    df.dropna(axis=0, inplace=True, how='any')
    return df

train_df = data_clean(train_df)
# 设置相同的index方便合并
train_df.set_index('vid',inplace=True)
test_df.set_index('vid', inplace=True)

In [63]:
train_tmp = pd.concat([train_df, tmp_nums], axis=1, join='inner').iloc[:,5:]
test_tmp = pd.concat([test_df, tmp_nums], axis=1, join='inner').iloc[:,5:]

In [69]:
# 清洗缺失值太多的人：
def clean_people(df, th):
    print('清理之前的人数：', df.shape[0])
    del_list = []
    for r in range(df.shape[0]):
        if(df.iloc[r,:].count() / df.shape[1] < th):
            del_list.append(df.index[r])
    print('清理的人数是：', len(del_list))
    df = df.drop(del_list, axis=0)
    print('清理后的人数是：', df.shape[0])
    return df

train_tmp = clean_people(train_tmp, 0.35)

清理之前的人数： 38192
清理的人数是： 9429
清理后的人数是： 28763


## 合并train和test
## 与主题向量结合之后，映射到0-100范围，做nmf的缺失值填充

In [75]:
# 合并
train_test_tmp = pd.concat([train_tmp, test_tmp], axis=0)

# 读取主题向量
topic_vec = pd.read_csv('./input/topic_vec20.csv')
topic_vec.set_index('Unnamed: 0', inplace=True)
print(topic_vec.shape)

# 与train_test合并
train_test_topic = pd.concat([train_test_tmp, topic_vec], axis=1, join='inner')
print(train_test_topic.shape)

(57298, 20)
(38301, 92)


### 缺失值补0，其他值映射到0-100范围

In [76]:
# 找最小值
def find_min(df, i):
    min_data = 100000
    for r in range(df.shape[0]):
        if (df.iloc[r, i] < min_data) and (df.iloc[r, i] != 0):
            min_data = df.iloc[r, i]
    return min_data

# 映射函数
class nmf_apply(object):
    def __init__(self, min_data, max_data, min_v, max_v):
        self.min_data = min_data
        self.max_data = max_data
        self.min_v = min_v
        self.max_v = max_v
        
    def set_min_max(self, min_data, max_data):
        self.min_data = min_data
        self.max_data = max_data
    
    def set_min_max_v(self, min_v, max_v):
        self.min_v = min_v
        self.max_v = max_v
    
    def nmf_app(self, x):
        if(x == 0):
            return x
        else:
            x = ((x - self.min_data) / (self.max_data - self.min_data)) * (self.max_v - self.min_v)
        return x

# 映射到 min_v 和 max_v 之间
def nmf_fx(df, min_v, max_v):
    nmf_i = nmf_apply(0, 0, min_v, max_v)
    
    for c in range(df.shape[1]):
        max_data = df.iloc[:, c].max()
        min_data = find_min(df, c)
        nmf_i.set_min_max(min_data, max_data)
        df.iloc[:, c] = df.iloc[:, c].apply(nmf_i.nmf_app)
    return df

In [77]:
tmp_nmf = train_test_topic.fillna(0)
tmp_nmf = nmf_fx(tmp_nmf, 0, 100)

In [79]:
tmp_nmf.to_csv('./input/tmp_nmf.txt', sep='\t', header=None, index=None)

In [80]:
print(tmp_nmf.shape)

(38301, 92)


# nmf补全向量与原矩阵结合：

In [133]:
# 读取nmf向量：
nmf_train_after = pd.read_csv('./input/train_set_nmf_res.txt', sep='\t', header=None)

(38301, 92)

In [ ]:
def apply_nmf(x):
    if x == 0:
        return 1
    else:
        return 0

mask = tmp_nmf.applymap(apply_nmf)
temp = mask.values*nmf_train_after.values + tmp_nmf.values
nmf_combine = pd.DataFrame(temp, index=tmp_nmf.index)

In [ ]:
# 标准化处理

def m_standard(df):
    for c in df.columns:
        c_std = df[c].std()
        c_mean = df[c].mean()
        df[c] = (df[c] - c_mean) / c_std
    return df

nmf_combine = m_standard(nmf_combine)

In [ ]:
# 储存
x_nmf = pd.concat([nmf_combine, train_df], axis=1, join='inner')
predict_nmf = pd.concat([test_df, nmf_combine], axis=1, join='inner').iloc[:,5:]

x_nmf.reset_index(inplace=True)
predict_nmf.reset_index(inplace=True)

x_nmf.to_csv('./input/new_nmf_train.csv', header=None, index=None)
predict_nmf.to_csv('./input/new_nmf_test.csv', header=None, index=None)

# 训练测试：

In [82]:
# 用均值填充 0

# 填充0的apply函数
def mean_apply(c_mean):
    def m_apply(x):
        if(x == 0):
            x = c_mean
        return x
    return m_apply

def count_not_zero(x):
    if (x == 0):
        return 0
    else:
        return 1
    
def f_mean(df):
    for c in range(df.shape[1]):
        c_mean = df.iloc[:,c].sum() / df.iloc[:,c].apply(count_not_zero).sum()
        df.iloc[:,c] = df.iloc[:,c].apply(mean_apply(c_mean))
    return df

tmp_fill_mean = f_mean(tmp_nmf)

In [85]:
# 做标准化处理：

def m_standard(df):
    for c in df.columns:
        c_std = df[c].std()
        c_mean = df[c].mean()
        df[c] = (df[c] - c_mean) / c_std
    return df

tmp_fill_mean = m_standard(tmp_fill_mean)

In [129]:
x_mean = pd.concat([tmp_fill_mean, train_df], axis=1, join='inner')
predict_mean = pd.concat([test_df, tmp_fill_mean], axis=1, join='inner').iloc[:,5:]

In [131]:
x_mean.reset_index(inplace=True)
predict_mean.reset_index(inplace=True)

x_mean.to_csv('./input/new_mean_train.csv', header=None, index=None)
predict_mean.to_csv('./input/new_mean_test.csv', header=None, index=None)

In [116]:
x_y_train = pd.concat([train_df, tmp_fill_mean], axis=1, join='inner')

x_train = x_y_train.iloc[:,5:].values
y_train = x_y_train.iloc[:,1].values
print(x_train.shape)
print(y_train.shape)

(28763, 92)
(28763,)


In [96]:
import xgboost as xgb
from xgboost.sklearn import XGBRegressor

In [117]:
import xgboost as xgb
from xgboost.sklearn import XGBRegressor

clf = XGBRegressor(
        objective='reg:linear',
        learning_rate=0.1,  # [默认是0.3]学习率类似，调小能减轻过拟合，经典值是0.01-0.2
        gamma=0,  # 在节点分裂时，只有在分裂后损失函数的值下降了，才会分裂这个节点。Gamma指定了节点分裂所需的最小损失函数下降值。这个参数值越大，算法越保守。
        subsample=0.8,  # 随机采样比例，0.5-1 小欠拟合，大过拟合
        colsample_bytree=0.8,  # 训练每棵树时用来训练的特征的比例
        reg_alpha=1,  # [默认是1] 权重的L1正则化项
        reg_lambda=1,  # [默认是1] 权重的L2正则化项
        max_depth=10,  # [默认是6] 树的最大深度，这个值也是用来避免过拟合的3-10
        min_child_weight=1,  # [默认是1]决定最小叶子节点样本权重和。当它的值较大时，可以避免模型学习到局部的特殊样本。但如果这个值过高，会导致欠拟合。
    )
dtrain = xgb.DMatrix(x_train, y_train)
xgb_params = clf.get_xgb_params()
clf_xgb = xgb.train(xgb_params, dtrain, num_boost_round=80)

In [119]:
x_predict = pd.concat([test_df, tmp_fill_mean], axis=1, join='inner').iloc[:, 5:].values
x_predict = xgb.DMatrix(x_predict)
res = clf_xgb.predict(x_predict)